In [1]:
!pip install pulp

In [7]:
import numpy as np

filename = "Tsp58_2.txt"
matrix = []
with open(filename, 'r') as file:
    for line in file:
        row = list(map(int, line.strip().split()))
        matrix.append(row)

m58 = np.zeros([58,58])
for i in range(0,len(m58)-1):
  for j in range(0,len(matrix[i])):
      m58[i][i+j+1]=matrix[i][j]

for i in range(0,len(m58)):
  for j in range(0,len(m58)):
    if i==j:
      m58[i][j]=99999
    elif i < j:
      m58[j][i]=m58[i][j]
print(m58)

[[99999.  2635.  2713. ...  3870.  1417.   739.]
 [ 2635. 99999.   314. ...  2072.  1196.  1517.]
 [ 2713.   314. 99999. ...  1882.  2699.  1557.]
 ...
 [ 3870.  2072.  1882. ... 99999.  2328.  2986.]
 [ 1417.  1196.  2699. ...  2328. 99999.   962.]
 [  739.  1517.  1557. ...  2986.   962. 99999.]]


In [8]:
size = 58
departure = [str(i) for i in range(size)]
arrival = [str(i) for i in range(size)]

distance = {}

for dep in departure:
    distance[dep] = {}
    for arr in arrival:
        cost = m58[int(dep)][int(arr)]
        distance[dep][arr] = cost

In [9]:
from pulp import *

lp_problem = LpProblem("Traveling Salesman Problem", LpMinimize)

routes = [ (d, a) for d in departure for a in arrival ]

vars = LpVariable.dicts("best route", (departure, arrival), 0, 1, LpInteger)
vars_f = LpVariable.dicts('Flow', (departure, arrival), 0, None, LpInteger)
n = len(departure)-1

lp_problem += (lpSum([vars[o][d] * distance[o][d] for (o, d) in routes]),"Soma_das_distancias_percorridas", )

for a in arrival:
    lp_problem += ( lpSum([vars[d][a] for d in departure]) == 1, f"Sum_Arrivel_{a}",)

for d in departure:
    lp_problem += ( lpSum([vars[d][a] for a in arrival]) == 1, f"Sum_departure_{d}",)

lp_problem += (
        lpSum([vars[a][a] for a in arrival]) == 0, f"Sum_{a,a}",)

for d in departure:
        for a in arrival:
                lp_problem += (
                        vars_f[d][a] <= n*vars[d][a]
                        )

for o in ['1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18','19','20','21','22','23','24',	'25','26','27',	'28','29','30','31','32','33','34','35','36','37','38','39','40','41','42','43','44','45','46','47','48','49','50','51','52','53','54','55','56','57']:
        lp_problem += (
                lpSum([vars_f[a][o]-vars_f[o][a] for a in arrival]) == 1,
                f"Flow_balance_{o}",
                )


In [ ]:
print("Status:", LpStatus[lp_problem.status])
lp_problem.solve()

Status: Not Solved


In [1]:
if lp_problem.status == LpStatusOptimal:
    print("Solução ótima encontrada:")
    for var in lp_problem.variables():
        print(var.name, "=", var.varValue)
else:
    print("Não foi possível encontrar uma solução ótima.")

NameError: name 'lp_problem' is not defined